In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 단계 1: 폰트 설치
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
!apt-get -qq -y install fonts-nanum > /dev/null
#fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

#font = fm.FontProperties(fname=fontpath, size=9)

#fm._rebuild()



fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설


In [ ]:
# 단계 2: 런타임 재시작
import os
os.kill(os.getpid(), 9)

In [ ]:
# 단계 3: 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/content/빅데이터이음.csv', encoding='utf-8')

print(df)


             일자         지역  문화취미-영화관      숙박       약국        연료  유통업영리 - 편의점  \
0    2025-01-01  인천광역시 강화군     61000  310100   501400   7369451      5430590   
1    2025-01-02  인천광역시 강화군     54000   62000  3396110  12740492      6514280   
2    2025-01-03  인천광역시 강화군      7000  240000  3793110  10325810      6818890   
3    2025-01-04  인천광역시 강화군     86000  250100  2719490  11967997      6222940   
4    2025-01-05  인천광역시 강화군      7000   20000   434900   5800303      4768610   
..          ...        ...       ...     ...      ...       ...          ...   
336  2025-01-27         기타         0       0        0         0        36150   
337  2025-01-28         기타         0       0        0         0       104950   
338  2025-01-29         기타         0       0        0         0       149650   
339  2025-01-30         기타         0       0        0         0        71200   
340  2025-01-31         기타         0       0        0         0        81910   

         일반휴게        학원        기타      

In [ ]:
df.columns

Index(['일자', '지역', '문화취미-영화관', '숙박', '약국', '연료', '유통업영리 - 편의점', '일반휴게', '학원',
       '기타', '전체금액'],
      dtype='object')

In [ ]:
df.head(1)

,일자,지역,문화취미-영화관,숙박,약국,연료,유통업영리 - 편의점,일반휴게,학원,기타,전체금액
0,2025-01-01,인천광역시 강화군,61000,310100,501400,7369451,5430590,36367190,398000,38417950,88855681


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 한글깨짐 방지
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')

# 1. 데이터 불러오기 + 합치기

# 파일 불러오기 (인코딩 문제 대비)
def load_file(path):
    try:
        return pd.read_csv(path, encoding='utf-8')
    except:
        return pd.read_csv(path, encoding='CP949')

# 파일 3개 읽기
df1 = load_file("bigdata.csv")
df2 = load_file("빅데이터이음2.csv")
df3 = load_file("빅데이터이음.csv")

# 합치기
df_data = pd.concat([df1, df2, df3], ignore_index=True)

# 날짜 형식 처리
df_data['일자'] = df_data['일자'].astype(str).str.replace('.', '-')
df_data['일자'] = pd.to_datetime(df_data['일자'])

# 월 / 군구 컬럼 추가
df_data['월'] = df_data['일자'].dt.to_period('M').astype(str)
df_data['군구'] = df_data['지역'].apply(lambda x: x.split()[-1])


# 2. 월별 분석


industry_cols = ['문화취미-영화관', '숙박', '약국', '연료', '유통업영리 - 편의점', '일반휴게', '학원', '기타']
group_cols = ['전체금액'] + industry_cols
months = ['2024-11', '2024-12', '2025-01']

monthly_data = {}

for m in months:
    df_m = df_data[df_data['월'] == m]

    # 업종 합계
    df_ind = df_m[industry_cols].sum().reset_index()
    df_ind.columns = ['업종', '총 소비 금액']
    df_ind = df_ind.sort_values(by='총 소비 금액', ascending=False)
    df_ind['총 소비 금액 (억 원)'] = (df_ind['총 소비 금액'] / 100000000).round(2)

    # 군구별 합계
    df_dist = df_m.groupby('군구')[group_cols].sum().reset_index()
    df_dist = df_dist.sort_values(by='전체금액', ascending=False)
    df_dist['전체금액 (억 원)'] = (df_dist['전체금액'] / 100000000).round(2)

    monthly_data[m] = {
        'industry': df_ind,
        'district': df_dist
    }

# 월별 전체 소비 요약
df_month_sum = df_data.groupby('월')[group_cols].sum().reset_index()
df_month_sum['전체금액 (억 원)'] = (df_month_sum['전체금액'] / 100000000).round(2)
df_month_sum.to_csv("월별 총 소비 금액.csv", index=False, encoding='utf-8-sig')


# 3. 시각화


# (1) 월별 업종별 소비 금액
for m in months:
    d = monthly_data[m]['industry']

    plt.figure(figsize=(12, 6))
    sns.barplot(x='업종', y='총 소비 금액 (억 원)', data=d)
    plt.title(f'업종별 총 소비 금액 ({m})')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'result1_{m}.png')
    plt.close()

# (2) 월별 총 소비 금액 비교
df_month_sum['월_sort'] = pd.to_datetime(df_month_sum['월'])
df_month_sum = df_month_sum.sort_values('월_sort')

plt.figure(figsize=(8, 6))
sns.lineplot(x='월', y='전체금액 (억 원)', data=df_month_sum, marker='o')
plt.title('월별 총 소비 금액 추이')
plt.tight_layout()
plt.savefig('result2.png')
plt.close()

# (3) 일별 소비 금액 비교
df_data['일'] = df_data['일자'].dt.day
df_daily = df_data.groupby(['월', '일'])['전체금액'].sum().reset_index()
df_daily['일별 총 소비 금액 (억 원)'] = df_daily['전체금액'] / 100000000

plt.figure(figsize=(16, 8))
sns.lineplot(x='일', y='일별 총 소비 금액 (억 원)', hue='월', data=df_daily, marker='o')
plt.title('일자별 총 소비 금액 추이')
plt.tight_layout()
plt.savefig('result3.png')
plt.close()

# (4) 군구별 업종 Stacked Bar (월별)
for m in months:
    df_dist = monthly_data[m]['district']
    df_plot = df_dist.set_index('군구')[industry_cols] / 100000000

    plt.figure(figsize=(15, 8))
    df_plot.plot(
        kind='bar',
        stacked=True,
        figsize=(15, 8),
        ax=plt.gca()
    )
    plt.title(f'군구별 업종별 소비 금액 ({m})')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'result4_{m}.png')
    plt.close()

FileNotFoundError: [Errno 2] No such file or directory: 'bigdata.csv'